# NEURON simulator

[NEURON simulator](https://nrn.readthedocs.io/en/8.2.6/) excels at simulating complex, multi-compartmental models with detailed biophysics. The NEURON simulator is widely used in the simulation of detailed neural mechanisms and networks of neurons in the neuroscience literature. The original programming language used to code the NEURON is Higher Order Calculator (HOC). NEURON may also be programmed in Python, which we will explore more in this series of writings.

## Installation of NEURON with GPU support (CoreNEURON)

### 1. Clone the latest version of NEURON

```{bash}
git clone https://github.com/neuronsimulator/nrn
cd nrn
```

### 2. Build

```{bash}
mkdir build
cd build
```

### 3. Load the modules

```{bash}
module load intel openmpi python cmake nvidia-hpc-sdk cuda  
```

In normal Linux systems, the modules are in the path `/usr/share/modules/modulefiles`. If the directory does not exists, then install using

```{bash}

sudo apt-get install environment-modules

# and configure the ~/.bashrc with 
source /etc/profile.d/modules.sh

```

### 4. Compile

Following compilation is for GPU architecture of 61

```{bash}
cmake .. \
      -DNRN_ENABLE_CORENEURON=ON \
      -DCORENRN_ENABLE_GPU=ON \
      -DNRN_ENABLE_INTERVIEWS=OFF \
      -DNRN_ENABLE_RX3D=OFF \
      -DCMAKE_INSTALL_PREFIX=$HOME/install \
      -DCMAKE_C_COMPILER=nvc \
      -DCMAKE_CXX_COMPILER=nvc++ \
      -DCMAKE_PREFIX_PATH=/usr/lib/x86_64-linux-gnu \
      -DCMAKE_CUDA_ARCHITECTURES="61" \  # need to know this
      -DCMAKE_EXE_LINKER_FLAGS: "-cuda -gpu=cuda12.6,lineinfo,cc61 -acc" # not necessary
      -DCMAKE_CXX_FLAGS="-O3 -g" \
      -DCMAKE_C_FLAGS="-O3 -g" \
      -DCMAKE_BUILD_TYPE=Custom
```

Check the CUDA_ARCHITECTURES via (nvidia-smi; assuming it is installed)

```{bash}
nvidia-smi --query-gpu=compute_cap --format=csv
```

### 5. Ringtest

Best way to check if the installation is working properly is by running the ring test as in

```{bash}
git clone https://github.com/nrnhines/ringtest.git
cd ringtest

nrnivmodl -coreneuron mod

# in any NEURON code add the following lines for CoreNEURON support (python files only)

h.cvode.cache_efficient(1)
if use_coreneuron:
    from neuron import coreneuron
    coreneuron.enable = True
    coreneuron.gpu = coreneuron_gpu
    
    
#run the three performance test 

# NEURON CPU Run
mpiexec -n 1 ./x86_64/special -mpi -python ringtest.py -tstop 10 -nring 128 -ncell 128 -branch 32 64

# CoreNEURON CPU Run
mpiexec -n 1 ./x86_64/special -mpi -python ringtest.py -tstop 10 -nring 128 -ncell 128 -branch 32 64 -coreneuron

# CoreNEURON GPU Run
mpiexec -n 1 ./x86_64/special -mpi -python ringtest.py -tstop 10 -nring 128 -ncell 128 -branch 32 64 -coreneuron -gpu
```

# Deep dentritic network (GPU support)

```{bash}

https://github.com/pkuzyc/DeepDendrite/tree/main


```